In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import os

In [43]:
#dataset
class BMIDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform = None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self):
        return len(self.annotations)
    def __getitem__(self,idx
                    ):
        img_path = self.annotations.iloc[idx, 2]
        image = Image.open(img_path).convert("RGB")
        bmi_label = float(self.annotations.iloc[idx, 7])

        if self.transform:
            image = self.transform(image)

        return (image, bmi_label)
    

In [44]:
resize = 224

In [45]:
transform = transforms.Compose([
    transforms.Resize((resize, resize)),  # Chọn kích thước ảnh sau khi augment
    transforms.ToTensor(),  # Chuyển ảnh sang tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Chuẩn hóa tensor
])

In [46]:
transform

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
    ToTensor()
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
)

In [47]:
class BMIModel(nn.Module):
    def __init__(self):
        super(BMIModel,self).__init__()
        self.conv1 = nn.Conv2d(3,6, 3) #224x224
        self.relu1 = nn.ReLU()
        self.batch1 = nn.BatchNorm2d(6)
        self.pool1 = nn.MaxPool2d( 2,2) #111x111
        
        self.conv2 = nn.Conv2d(6 ,10 , 3) # 110 x110
        self.relu2 = nn.ReLU()
        self.batch2 = nn.BatchNorm2d(10)
        self.pool2 = nn.MaxPool2d( 2,2)# 55x55
        
        self.l_conv1 = int((224- 3) + 1)
        self.l_maxpool1= int((self.l_conv1 - 2)/2 + 1)

        self.l_conv2 = int((self.l_maxpool1- 3) + 1)
        self.l_maxpool2= int((self.l_conv2 - 2)/2 + 1)        
        self.linear = nn.Linear(self.l_maxpool2*self.l_maxpool2*10,1)
    
    def forward(self,x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.batch1(x)
        x = self.pool1(x)
        
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.batch2(x)
        x = self.pool2(x)
        x = torch.flatten(x, start_dim=1)
        x = self.linear(x)
        
        return x

In [48]:
dataset = BMIDataset(csv_file ='/home/quanhhh/Documents/pre_bmi/pre_databmi.csv',
                     root_dir = '/home/quanhhh/Downloads/DATA_BMI/height_weight/',
                     transform = transform )

In [49]:
# annotations = pd.read_csv('/home/quanhhh/Documents/pre_bmi/pre_databmi.csv')
# img_path = annotations.iloc[1, 0]

In [50]:
#set up data loader and transformations
total_samples = len(dataset)
total_samples
from torch.utils.data import random_split
total_samples = len(dataset)
train_size = int(0.7 * total_samples)  # Adjust the split ratio as needed
test_size = total_samples - train_size

train_set, test_set = random_split(dataset, [train_size, test_size])

In [51]:
batch_size = 226


In [52]:
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)


In [53]:
model = BMIModel()


In [56]:
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps
        
    def forward(self,yhat,y):
        loss = torch.sqrt(self.mse(yhat,y) + self.eps)
        return loss

In [59]:
criterion = RMSELoss()  # Root Mean Squared Error Loss for regression tasks
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [60]:
from sklearn.metrics import accuracy_score


In [61]:
def train(model, train_loader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        for image, bmi_labels in train_loader:
            optimizer.zero_grad()
            outputs = model(image)

            loss = criterion(outputs, bmi_labels.view(-1, 1).float())/len(train_loader)  # Convert to float
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')


In [62]:
def evaluate(model, test_loader, criterion):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            # Forward pass
            outputs = model(inputs)
            outputs = torch.squeeze(outputs)
            # Compute the loss (optional, for tracking purposes)
            loss = criterion(outputs, labels)
            

    # Compute metrics (example: accuracy)
    # accuracy = accuracy_score(all_labels, all_preds)
    print(f'Loss: {loss/len(test_loader)}')

In [63]:
train(model, train_loader, criterion, optimizer, num_epochs=100)

Epoch 1/100, Loss: 24.255720138549805
Epoch 2/100, Loss: 17.765771865844727
Epoch 3/100, Loss: 11.87490463256836
Epoch 4/100, Loss: 8.754712104797363
Epoch 5/100, Loss: 9.622421264648438
Epoch 6/100, Loss: 10.018963813781738
Epoch 7/100, Loss: 8.825141906738281
Epoch 8/100, Loss: 7.0407819747924805
Epoch 9/100, Loss: 5.773986339569092
Epoch 10/100, Loss: 5.429225444793701
Epoch 11/100, Loss: 5.363011360168457
Epoch 12/100, Loss: 5.188183307647705
Epoch 13/100, Loss: 4.8243231773376465
Epoch 14/100, Loss: 4.281974792480469
Epoch 15/100, Loss: 3.5895016193389893
Epoch 16/100, Loss: 2.9358675479888916
Epoch 17/100, Loss: 2.6316113471984863
Epoch 18/100, Loss: 2.8169877529144287
Epoch 19/100, Loss: 2.927126884460449
Epoch 20/100, Loss: 2.584408760070801
Epoch 21/100, Loss: 1.8272849321365356
Epoch 22/100, Loss: 1.5545367002487183
Epoch 23/100, Loss: 2.0140390396118164
Epoch 24/100, Loss: 1.9215281009674072
Epoch 25/100, Loss: 1.2952656745910645
Epoch 26/100, Loss: 1.3137915134429932
Epoch 

In [ ]:
evaluate(model,test_loader,criterion)

In [ ]:
def train():
    # train_epoch
    # test_epoch
    # luu model/ tra ve model
    # self.best_model = copy.deepcopy(self.model)
    pass


